In [1]:
import matplotlib.pyplot as plt
import urllib.request
import os
import gzip
import struct
import pandas as pd
import numpy as np
from sklearn.datasets import load_digits
from sklearn.model_selection import cross_val_score

from DBNAC import DBNClassifier, SimpleDBNClassifier
data_folder = os.path.join(os.getcwd(), 'data')
os.makedirs(data_folder, exist_ok=True)

# urllib.request.urlretrieve('https://azureopendatastorage.blob.core.windows.net/mnist/train-images-idx3-ubyte.gz',
#                            filename=os.path.join(data_folder, 'train-images.gz'))
# urllib.request.urlretrieve('https://azureopendatastorage.blob.core.windows.net/mnist/train-labels-idx1-ubyte.gz',
#                            filename=os.path.join(data_folder, 'train-labels.gz'))
# urllib.request.urlretrieve('https://azureopendatastorage.blob.core.windows.net/mnist/t10k-images-idx3-ubyte.gz',
#                            filename=os.path.join(data_folder, 'test-images.gz'))
# urllib.request.urlretrieve('https://azureopendatastorage.blob.core.windows.net/mnist/t10k-labels-idx1-ubyte.gz',
#                            filename=os.path.join(data_folder, 'test-labels.gz'))
# load compressed MNIST gz files and return numpy arrays
def load_data(filename, label=False):
    with gzip.open(filename) as gz:
        struct.unpack('I', gz.read(4))
        n_items = struct.unpack('>I', gz.read(4))
        if not label:
            n_rows = struct.unpack('>I', gz.read(4))[0]
            n_cols = struct.unpack('>I', gz.read(4))[0]
            res = np.frombuffer(gz.read(n_items[0] * n_rows * n_cols), dtype=np.uint8)
            res = res.reshape(n_items[0], n_rows * n_cols)
        else:
            res = np.frombuffer(gz.read(n_items[0]), dtype=np.uint8)
            res = res.reshape(n_items[0], 1)
    return res

In [2]:
# note we also shrink the intensity values (X) from 0-255 to 0-1. This helps the model converge faster.
X_train = load_data(os.path.join(
    data_folder, 'train-images.gz'), False)
X_test = load_data(os.path.join(data_folder, 'test-images.gz'), False)
y_train = load_data(os.path.join(
    data_folder, 'train-labels.gz'), True).reshape(-1)
y_test = load_data(os.path.join(
    data_folder, 'test-labels.gz'), True).reshape(-1)
X_train = (X_train > 127)*1
X_test = (X_test > 127)*1

In [3]:
def acc(pred, y):
    return np.mean((pred==y)*1.0)

In [13]:
config_2 = [[700,700,10]]
for config in config_2:
    clf = SimpleDBNClassifier(n_hiddens=config, lr_pre_train=1e-5, lr_fine_tune=0.01, 
                    epochs_pre_train=0, epochs_fine_tune=100, batch_size=50, k=3, 
                    random_state=42, use_gpu=True, verbose=False)

    clf.fit(X_train,y_train)
    print('config=',config)
    print('Accuracy train', acc(clf.predict(X_train), y_train))
    print('Accuracy test', acc(clf.predict(X_test), y_test))

KeyboardInterrupt: 

In [ ]:
n_hiddens = [[200, 200, 10],[200,200, 200, 10],[200, 200, 200, 200, 10]]
config = [1000]
for i in n_hiddens:
    clf = SimpleDBNClassifier(n_hiddens=i, lr_pre_train=0.1, lr_fine_tune=0.1, 
                    epochs_pre_train=20, epochs_fine_tune=20, batch_size=50, k=3, 
                    random_state=42, use_gpu=True, verbose=False)
    # scores = cross_val_score(clf, X_train, y_train)
    # print(np.mean(scores))
    # print(scores)
    x = X_train[:]
    y = y_train[:]
    clf.fit(x,y)
    print('i=',i)
    print('Accuracy train', acc(clf.predict(x), y))
    print('Accuracy test', acc(clf.predict(X_test), y_test))

i= [200, 200, 10]
Accuracy train 0.96825
Accuracy test 0.9506
i= [200, 200, 200, 10]
Accuracy train 0.9681166666666666
Accuracy test 0.9557
i= [200, 200, 200, 200, 10]
Accuracy train 0.9649833333333333
Accuracy test 0.9509
